# Prediction of potential risk of coronary heart disease

In [31]:
from pyspark import Row

from pyspark.sql import SparkSession
from pyspark.sql.functions import corr

from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

spark = SparkSession.builder.appName("heart_disease").getOrCreate()

df = spark.read.csv(path="data_cardiovascular_risk.csv", inferSchema=True, header=True)

df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- education: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- is_smoking: string (nullable = true)
 |-- cigsPerDay: double (nullable = true)
 |-- BPMeds: double (nullable = true)
 |-- prevalentStroke: integer (nullable = true)
 |-- prevalentHyp: integer (nullable = true)
 |-- diabetes: integer (nullable = true)
 |-- totChol: double (nullable = true)
 |-- sysBP: double (nullable = true)
 |-- diaBP: double (nullable = true)
 |-- BMI: double (nullable = true)
 |-- heartRate: double (nullable = true)
 |-- glucose: double (nullable = true)
 |-- TenYearCHD: integer (nullable = true)



### Count rows before drop null rows

In [32]:
print(df.count())

3390


In [33]:
droped = df.dropna()

### Count rows after drop null rows

In [34]:
print(droped.count())

2927


### Replace categorical columns with numeric columns

In [35]:
indexer = StringIndexer(inputCols=['sex', 'is_smoking'], outputCols=['sex_cat', 'is_smoking_cat'])
indexed = indexer.fit(droped).transform(droped)
indexed.select(['sex', 'is_smoking', 'sex_cat', 'is_smoking_cat' ]).show(10)

+---+----------+-------+--------------+
|sex|is_smoking|sex_cat|is_smoking_cat|
+---+----------+-------+--------------+
|  M|        NO|    1.0|           0.0|
|  F|       YES|    0.0|           1.0|
|  M|       YES|    1.0|           1.0|
|  F|       YES|    0.0|           1.0|
|  F|        NO|    0.0|           0.0|
|  M|        NO|    1.0|           0.0|
|  M|       YES|    1.0|           1.0|
|  F|        NO|    0.0|           0.0|
|  F|        NO|    0.0|           0.0|
|  M|       YES|    1.0|           1.0|
+---+----------+-------+--------------+
only showing top 10 rows



### Create features Vector

In [36]:
assembler = VectorAssembler(inputCols=['age', 'education', 'cigsPerDay', 'BPMeds', 'prevalentStroke',
                             'prevalentHyp', 'diabetes', 'totChol', 'sysBP', 'diaBP', 'BMI',
                             'heartRate', 'glucose', 'sex_cat', 'is_smoking_cat'],
                            outputCol="features")
output = assembler.transform(indexed)
final_data = output.select(["features", "TenYearCHD"])
final_data.show(10)

+--------------------+----------+
|            features|TenYearCHD|
+--------------------+----------+
|[36.0,4.0,0.0,0.0...|         0|
|[46.0,1.0,10.0,0....|         0|
|[50.0,1.0,20.0,0....|         1|
|[64.0,1.0,30.0,0....|         0|
|[61.0,3.0,0.0,0.0...|         1|
|[61.0,1.0,0.0,0.0...|         0|
|[36.0,4.0,35.0,0....|         0|
|(15,[0,1,7,8,9,10...|         0|
|(15,[0,1,7,8,9,10...|         0|
|[44.0,1.0,40.0,0....|         0|
+--------------------+----------+
only showing top 10 rows



### Divide the data to train and test data and check distribution

In [37]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

train_data.describe().show()

test_data.describe().show()

+-------+-------------------+
|summary|         TenYearCHD|
+-------+-------------------+
|  count|               2089|
|   mean|0.15461943513642892|
| stddev|0.36162807838192257|
|    min|                  0|
|    max|                  1|
+-------+-------------------+

+-------+-------------------+
|summary|         TenYearCHD|
+-------+-------------------+
|  count|                838|
|   mean|0.14439140811455847|
| stddev|  0.351696077552079|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



### Train data and evaluete test data

In [38]:
disease_ls = LogisticRegression(featuresCol='features', labelCol="TenYearCHD")

trained_disease_model = disease_ls.fit(train_data)

prediction_results = trained_disease_model.transform(test_data)

### Check results

In [39]:
eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='TenYearCHD')
eval.evaluate(prediction_results)

0.5564046705165001

### The most correlation column with TenYearCHD

In [40]:
indexed.select(corr(col1='TenYearCHD', col2='age')).show()

+---------------------+
|corr(TenYearCHD, age)|
+---------------------+
|  0.23442278429915242|
+---------------------+



### Test with real data

In [41]:
real_data = [Row(age=29, education=4.0,cigsPerDay=0.0, BPMeds=0.0, prevalentStroke=0,
                prevalentHyp=1, diabetes=0, totChol=250.0, sysBP=130.0, diaBP=80.0,
                BMI=26.2, heartRate=72.0, glucose=87, sex_cat=1.0, is_smoking_cat=0.0),
             Row(age=64, education=4.0,cigsPerDay=10.0, BPMeds=1.0, prevalentStroke=1,
                prevalentHyp=1, diabetes=1, totChol=300.0, sysBP=146.0, diaBP=90.0,
                BMI=37.0, heartRate=100.0, glucose=190, sex_cat=1.0, is_smoking_cat=1.0),
             ]
real_df = spark.createDataFrame(real_data)
transform_real_data = assembler.transform(real_df)
real_data = transform_real_data.select(["features"])

real_prediction = trained_disease_model.transform(real_data)
real_prediction.show()

+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|[29.0,4.0,0.0,0.0...|[3.20383565554092...|[0.96097836484015...|       0.0|
|[64.0,4.0,10.0,1....|[-2.0076461659550...|[0.11840245820434...|       1.0|
+--------------------+--------------------+--------------------+----------+



In [42]:
spark.stop()